# Encoder

<div class="alert alert-info">

This tutorial is available as an IPython notebook at [Malaya/example/augmentation-encoder](https://github.com/huseinzol05/Malaya/tree/master/example/augmentation-encoder).
    
</div>

In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
%%time

import malaya

CPU times: user 3.22 s, sys: 3.29 s, total: 6.51 s
Wall time: 2.31 s


### Why augmentation

Let say you have a very limited labelled corpus, and you want to add more, but labelling is very costly.

So, text augmentation! We provided few augmentation interfaces in Malaya.

In [3]:
string = 'saya suka makan ayam dan ikan'
text = 'Perdana Menteri berkata, beliau perlu memperoleh maklumat terperinci berhubung isu berkenaan sebelum kerajaan dapat mengambil sebarang tindakan lanjut. Bagaimanapun, beliau yakin masalah itu dapat diselesaikan dan pentadbiran kerajaan boleh berfungsi dengan baik.'

In [4]:
tokenizer = malaya.preprocessing.Tokenizer()

### Load Wordvector

dictionary of synonym is quite hard to populate, required some domain experts to help us. So we can use wordvector to find nearest words.

```python
def wordvector(
    string: str,
    wordvector,
    threshold: float = 0.5,
    top_n: int = 5,
    soft: bool = False,
):
    """
    augmenting a string using wordvector.

    Parameters
    ----------
    string: str
    wordvector: object
        wordvector interface object.
    threshold: float, optional (default=0.5)
        random selection for a word.
    soft: bool, optional (default=False)
        if True, a word not in the dictionary will be replaced with nearest jarowrinkler ratio.
        if False, it will throw an exception if a word not in the dictionary.
    top_n: int, (default=5)
        number of nearest neighbors returned. Length of returned result should as top_n.

    Returns
    -------
    result: List[str]
    """
```

In [5]:
vocab_wiki, embedded_wiki = malaya.wordvector.load(model = 'wikipedia')
word_vector_wiki = malaya.wordvector.WordVector(embedded_wiki, vocab_wiki)

Load pretrained wordvector into `malaya.wordvector.WordVector` class will disable eager execution.
2022-12-12 13:17:25.893504: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-12 13:17:25.897836: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-12-12 13:17:25.897855: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: husein-MS-7D31
2022-12-12 13:17:25.897859: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: husein-MS-7D31
2022-12-12 13:17:25.897923: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: Not found: was una

In [6]:
malaya.augmentation.encoder.wordvector(
    ' '.join(tokenizer.tokenize(string)), word_vector_wiki, soft = True
)

2022-12-12 13:17:25.918738: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 781670400 exceeds 10% of free system memory.
2022-12-12 13:17:26.159773: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 781670400 exceeds 10% of free system memory.
2022-12-12 13:17:26.165027: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 781670400 exceeds 10% of free system memory.
2022-12-12 13:17:26.261865: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 781670400 exceeds 10% of free system memory.


['saya suka makan ayam dan ikan',
 'kamu suka minum ayam dan ayam',
 'anda suka tidur ayam dan ular',
 'kami suka mandi ayam dan keju',
 'aku suka berehat ayam dan lembu']

In [7]:
malaya.augmentation.encoder.wordvector(
    ' '.join(tokenizer.tokenize(text)), word_vector_wiki, soft = True
)

2022-12-12 13:17:26.395871: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 781670400 exceeds 10% of free system memory.


['Perdana Menteri berkata , beliau perlu memperoleh maklumat terperinci berhubung isu berkenaan sebelum kerajaan dapat mengambil sebarang tindakan lanjut . Bagaimanapun , beliau yakin masalah itu dapat diselesaikan dan pentadbiran kerajaan boleh berfungsi dengan baik .',
 'Perdana Menteri berkata , dia perlu mendapatkan data terperinci berhubung isu berkaitan sebelum kerajaan dapat mendapat segala tindakan terperinci . Bagaimanapun , dia yakin gangguan tersebut boleh diselesaikan dan pentadbiran kerajaan dapat berfungsi dengan sempurna .',
 'Perdana Menteri berkata , baginda perlu memperolehi bacaan terperinci berhubung isu tertentu sebelum kerajaan dapat menghabiskan sesuatu tindakan lanjutan . Bagaimanapun , baginda yakin kelemahan ini harus diselesaikan dan pentadbiran kerajaan harus berfungsi dengan kuat .',
 'Perdana Menteri berkata , mereka perlu meraih penjelasan terperinci berhubung isu tersebut sebelum kerajaan dapat mengubah suatu tindakan ringkas . Bagaimanapun , mereka yaki

### Load Transformer

Problem with wordvector, it just replaced a word for near synonym without understood the whole sentence context, so, Transformer comes to the rescue!

```python
def transformer(
    string: str,
    model,
    threshold: float = 0.5,
    top_p: float = 0.9,
    top_k: int = 100,
    temperature: float = 1.0,
    top_n: int = 5,
):

    """
    augmenting a string using transformer + nucleus sampling / top-k sampling.

    Parameters
    ----------
    string: str
    model: object
        transformer interface object. Right now only supported BERT, ALBERT and ELECTRA.
    threshold: float, optional (default=0.5)
        random selection for a word.
    top_p: float, optional (default=0.8)
        cumulative sum of probabilities to sample a word. 
        If top_n bigger than 0, the model will use nucleus sampling, else top-k sampling.
    top_k: int, optional (default=100)
        k for top-k sampling.
    temperature: float, optional (default=0.8)
        logits * temperature.
    top_n: int, (default=5)
        number of nearest neighbors returned. Length of returned result should as top_n.

    Returns
    -------
    result: List[str]
    """
```

In [8]:
malaya.transformer.available_transformer()

,Size (MB),Description
bert,425.6,Google BERT BASE parameters
tiny-bert,57.4,Google BERT TINY parameters
albert,48.6,Google ALBERT BASE parameters
tiny-albert,22.4,Google ALBERT TINY parameters
xlnet,446.6,Google XLNET BASE parameters
alxlnet,46.8,Malaya ALXLNET BASE parameters
electra,443,Google ELECTRA BASE parameters
small-electra,55,Google ELECTRA SMALL parameters


In [9]:
electra = malaya.transformer.load(model = 'electra')

Instructions for updating:
Use `tf.random.categorical` instead.
INFO:tensorflow:Restoring parameters from /home/husein/Malaya/electra-model/base/electra-base/model.ckpt


In [10]:
bert = malaya.transformer.load(model = 'bert')

Downloading:   0%|          | 0.00/413M [00:00<?, ?B/s]

INFO:tensorflow:Restoring parameters from /home/husein/Malaya/bert-model/base/bert-base-v3/model.ckpt


In [11]:
albert = malaya.transformer.load(model = 'albert')

Downloading:   0%|          | 0.00/45.4M [00:00<?, ?B/s]

INFO:tensorflow:Restoring parameters from /home/husein/Malaya/albert-model/base/albert-base/model.ckpt


In [13]:
malaya.augmentation.encoder.transformer(' '.join(tokenizer.tokenize(string)), electra)

2022-12-12 13:18:23.815395: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:689] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" vendor: "GenuineIntel" model: "103" frequency: 2112 num_cores: 20 environment { key: "cpu_instruction_set" value: "AVX SSE, SSE2, SSE3, SSSE3, SSE4.1, SSE4.2" } environment { key: "eigen" value: "3.3.90" } l1_cache_size: 49152 l2_cache_size: 1310720 l3_cache_size: 26214400 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


['saya suka ada telur dan ikan',
 'saya suka mie ayam dan ikan',
 'saya suka sop ikan dan ikan',
 'saya suka makanan telur dan ikan',
 'saya suka masakan telur dan ikan']

In [14]:
malaya.augmentation.encoder.transformer(' '.join(tokenizer.tokenize(string)), bert)

2022-12-12 13:18:28.479227: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:689] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" vendor: "GenuineIntel" model: "103" frequency: 2112 num_cores: 20 environment { key: "cpu_instruction_set" value: "AVX SSE, SSE2, SSE3, SSSE3, SSE4.1, SSE4.2" } environment { key: "eigen" value: "3.3.90" } l1_cache_size: 49152 l2_cache_size: 1310720 l3_cache_size: 26214400 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


['Gue suka sate ayam dan telur',
 'Aku suka lemak ayam dan sayur',
 'Aku suka ketupat ayam dan sayur',
 'Aku suka makan ayam dan sayur',
 'Saya suka sambal ayam dan sayur']

In [15]:
malaya.augmentation.encoder.transformer(' '.join(tokenizer.tokenize(string)), albert)

2022-12-12 13:18:30.721373: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:689] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" vendor: "GenuineIntel" model: "103" frequency: 2112 num_cores: 20 environment { key: "cpu_instruction_set" value: "AVX SSE, SSE2, SSE3, SSSE3, SSE4.1, SSE4.2" } environment { key: "eigen" value: "3.3.90" } l1_cache_size: 49152 l2_cache_size: 1310720 l3_cache_size: 26214400 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


['Yang saat makan ayam dan kucing',
 'Aku gemar makan ayam dan anjing',
 'kadang tidak makan pepaya dan rendang',
 'Aku harus makan kecoa dan sayur',
 'lebih mau makan nasi dan mekdi']

In [16]:
malaya.augmentation.encoder.transformer(' '.join(tokenizer.tokenize(text)), electra)

['Perdana Menteri berkata , beliau perlu memperoleh perhatian terperinci berhubung isu ini sebelum beliau gagal mengambil sebarang tindakan serius . Bagaimanapun , beliau berharap masalah ini dapat diselesaikan dan integriti kelak perlu diselesaikan dengan baik .',
 'Perdana Menteri berkata , beliau perlu memperoleh maklumat terperinci berhubung isu berkenaan sebelum kerajaan cuba mengambil sebarang tindakan sewajarnya . Bagaimanapun , beliau berharap permasalahan itu dapat diselesaikan dan mungkin PH boleh ditangani dengan baik .',
 'Perdana Menteri berkata , beliau perlu memperoleh maklumat terperinci berhubung masalah berkenaan sebelum tidak mempertimbangkan mengambil sebarang tindakan susulan . Bagaimanapun , beliau berharap masalah berkenaan dapat diselesaikan dan pentadbiran kerajaan perlu diselesaikan dengan teliti .',
 'Perdana Menteri berkata , beliau perlu memperoleh maklumat terperinci berhubung perkara tersebut sebelum kementerian PH mengambil sebarang tindakan lanjut . Bag

In [17]:
malaya.augmentation.encoder.transformer(' '.join(tokenizer.tokenize(text)), electra)

['Perdana Menteri berkata , beliau perlu memperoleh maklumat terperinci berhubung isu 1MDB sebelum kerajaan dapat mengambil sebarang tindakan lanjut . Bagaimanapun , beliau yakin perkara ini dapat diselesaikan kerana pentadbiran kerajaan seharusnya berfungsi dengan baik .',
 'Perdana Menteri berkata , beliau perlu memperoleh maklumat terperinci berhubung isu ini sebelum kerajaan dapat mengambil sebarang tindakan lanjut . Bagaimanapun , beliau yakin perkara 1MDB dapat diselesaikan sekiranya pentadbiran kerajaan dapat berfungsi dengan baik .',
 'Perdana Menteri berkata , beliau perlu memperoleh laporan terperinci berhubung isu itu sebelum kerajaan dapat mengambil sebarang tindakan lanjut . Bagaimanapun , beliau yakin isu ini dapat diselesaikan kerana pentadbiran kerajaan akan berfungsi dengan baik .',
 'Perdana Menteri berkata , beliau perlu memperoleh maklumat terperinci berhubung isu itu sebelum kerajaan dapat mengambil sebarang tindakan lanjut . Bagaimanapun , beliau yakin masalah itu

In [18]:
malaya.augmentation.encoder.transformer(' '.join(tokenizer.tokenize(text)), albert)

['Perdana Menteri berkata, beliau akan memberikan maklumat lanjut berhubung isu teknikal agar beliau dapat berhubung sebarang tindakan lanjut. Bagaimanapun, beliau berharap masalah itu dapat diselesaikan dan pentadbirannya tetap diselesaikan dengan baik.',
 'Perdana Menteri berkata, beliau tidak mempunyai maklumat lanjut berhubung isu ini kerana tidak dapat menghalang sebarang tindakan lanjut. Bagaimanapun, beliau berharap masalah itu dapat diselesaikan kerana pentadbiran beliau akan diselesaikan dengan baik.',
 'Perdana Menteri berkata, beliau tidak memberikan maklumat lanjut berhubung isu teknikal dan tidak dapat mengambil sebarang tindakan lanjut. Bagaimanapun, beliau berharap masalah itu dapat diselesaikan agar pentadbiran tidak akan pulih dengan baik.',
 'Perdana Menteri berkata, beliau tidak memberi maklumat lanjut berhubung isu ini dan ini dapat mengeluarkan sebarang tindakan lanjut. Bagaimanapun, beliau yakin masalah itu dapat diselesaikan dan pentadbiran akan tetap pulih denga